In [1]:
import pandas as pd
import os,json,glob,pickle
import sys
import re

In [2]:
os.chdir('./data')

In [24]:
car_base = pd.read_excel('car_base.xlsx')

In [25]:
car_base.sample(5)

,model,brand,seg,title,duration,price_range,recall_num
487,3568,푸조,준중형,308,2018.02 ~ 현재,3252~3946,0
510,3593,폭스바겐,중형,Passat 저공해3종,2018.08 ~ 현재,3614,0
408,3405,링컨,준대형,MKZ 저공해2종,2016.09 ~ 현재,4707~5910,1
142,2559,볼보,대형SUV,XC90(1세대),2003.07 ~ 2015.04,7030~7450,1
333,3273,현대,중형SUV,싼타페 더 프라임(DM),2015.06 ~ 2018.08,2695~4035,0


In [26]:
car_base.model = car_base.model.astype('str')

In [108]:
car_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 7 columns):
model          526 non-null int64
brand          526 non-null object
seg            526 non-null object
title          526 non-null object
launching      526 non-null object
price_range    502 non-null object
recall_num     526 non-null int64
dtypes: int64(2), object(5)
memory usage: 28.8+ KB


In [74]:
#중고차데이터 합치기
json_files = glob.glob('usedCar_mpark_*.json')

used_car_data = {}
for json_file in json_files:
    with open(json_file) as fp:
        used_car_data.update(json.load(fp))

In [31]:
len(used_car_data)

7361

In [32]:
used_car_data['9011500347'] # 데이터 다있는 경우

{'title': '기아 카니발R ',
 'sub_title': '뉴카니발리무진GLX',
 'owned': '1인소유',
 'info': ['2011  년식', '68,720km', '자동', '경유'],
 'price': '1,420만원',
 '차량번호': '60보7792',
 '배기량': '2,199 CC',
 '연식': '2011 ',
 '색상': '검정',
 '주행거리': '68,720 km',
 '압류/저당': '정보없음/정보없음',
 '변속기': '자동',
 '세금미납': ' - ',
 '연료': '경유',
 '제시번호': ' 2019001542 ',
 '용도변경 이력': '없음',
 '차량번호/소유자변경': '0회 /1회',
 '특수 사고이력': '전손 : 0 / 침수 전손 : 0 / 침수 분손 : 0 / 도난 : 0',
 '보험사고(내차 피해)': '3회 (1,624,568원)',
 '보험사고(타차 가해)': '1회 (1,199,870원)',
 '점검일시': '2019-01-09',
 '점검자': '박기현',
 '동일성확인 ': '양호',
 '불법구조변경': '',
 '사고유무 ': '무',
 '침수유무': '무',
 '자동차의 상태': '교환 0건 / 부식 0건 / 판금·용접 0건',
 'option_0': ['네비게이션(매립)', '후방카메라', '썬루프(일반형)', '블랙박스'],
 'option_1': ['전자동 에어컨', '알루미늄휠', '가죽시트'],
 'option_2': ['에어백(운전석)',
  '에어백(동반석)',
  '에어백(사이드)',
  '에어백(커튼)',
  'ABS',
  'TCS',
  'ESP/VDC'],
 'option_3': ['ECM 룸미러', '하이패스', '후방감지센서', '크루즈컨트롤'],
 'option_4': ['열선핸들', '오토 슬라이딩도어'],
 '판매자설명': '★완무★1인신조실키로수★오슬,썬룹,정품네비후방등등'}

json인 used_car_data를 pandas로 만들자

In [38]:
used_car_df = {'demoNo':[],'title':[],'sub_title':[], 'baegi':[],'year_old':[],'color':[], 'driven_dist':[], 'trans':[], 'fuel':[]
               ,'mortgage':[], 'tax_to_pay':[], 'ch_purpose':[],'ch_num':[],'ch_owner':[]
               ,'total_loss':[], 'total_flood':[], 'part_flood':[], 'theft':[],'damaged':[], 'damaging':[]
               ,'opt_prefer':[],'opt_basic':[], 'opt_safe':[], 'opt_conven':[], 'opt_etc':[]
               ,'description':[], 'price':[]}

In [39]:
key_exceptions =[]
for k,v in used_car_data.items():
    if v == {}:
        continue
    try:
        used_car_df['demoNo'].append(k)
        used_car_df['title'].append(v['title'])
        used_car_df['sub_title'].append(v['sub_title'])

        used_car_df['baegi'].append(re.sub('[\D]','',v['배기량']))
        used_car_df['year_old'].append(v['연식'])
        used_car_df['color'].append(v['색상'])
        used_car_df['driven_dist'].append(re.sub('[\D]','',v['주행거리']))
        used_car_df['trans'].append(v['변속기'])
        used_car_df['fuel'].append(v['연료'])

        used_car_df['mortgage'].append(v['압류/저당'])
        used_car_df['tax_to_pay'].append(v['세금미납'])
        used_car_df['ch_purpose'].append(v['용도변경 이력'])

        ch_num, ch_owner = [el.strip() for el in v['차량번호/소유자변경'].split('/')]
        used_car_df['ch_num'].append(ch_num)
        used_car_df['ch_owner'].append(ch_owner)

        total_loss,total_flood,part_flood,theft  = [re.sub('[\D]','',el) for el in v['특수 사고이력'].split('/')]
        used_car_df['total_loss'].append(total_loss)
        used_car_df['total_flood'].append(total_flood)
        used_car_df['part_flood'].append(part_flood)
        used_car_df['theft'].append(theft)
    
    
        p = re.compile(r'(\S*원)')
        used_car_df['damaged'].append(re.sub('\D','',re.search(p,v['보험사고(내차 피해)']).group())) 
        used_car_df['damaging'].append(re.sub('\D','',re.search(p,v['보험사고(타차 가해)']).group()))  

        allopt_0 = ['네비게이션(매립)', '후방카메라', '썬루프(일반형)','파노라마 썬루프','버튼시동/스마트키', '블랙박스','4WD(4륜구동)','스톱앤고(ISG)']
        allopt_1 = ['전자동 에어컨', '알루미늄휠','HID/LED 헤드램프','가죽시트','전동시트','열선시트','통풍시트','CD/MP3 플레이어']
        allopt_2 = ['에어백(운전석)','에어백(동반석)','에어백(사이드)','에어백(커튼)','ABS','TCS','ESP/VDC','TPMS(타이어공기압)']
        allopt_3 = ['내비게이션(거치)','ECM 룸미러', '하이패스', '후방감지센서','전방감지센서', '크루즈컨트롤', '전자식 파킹브레이크','SPAS(자동주차)']
        allopt_4 = ['열선핸들','뒷좌석 TV', '오토 슬라이딩도어', '파워 트렁크','사이드스텝','HUD(헤드업디스플레이)','차선이탈 경보시스템','후측방 경보시스템(BSD)']
        
        opt_0 = [0 for i in range(8)]
        for opt in v['option_0']:
            opt_0[allopt_0.index(opt)] = 1
        used_car_df['opt_prefer'].append(opt_0)
        
        opt_1 = [0 for i in range(8)]
        for opt in v['option_1']:
            opt_1[allopt_1.index(opt)] = 1
        used_car_df['opt_basic'].append(opt_1)

        opt_2 = [0 for i in range(8)]
        for opt in v['option_2']:
            opt_2[allopt_2.index(opt)] = 1
        used_car_df['opt_safe'].append(opt_2)
        
        
        opt_3 = [0 for i in range(8)]
        for opt in v['option_3']:
            opt_3[allopt_3.index(opt)] = 1
        used_car_df['opt_conven'].append(opt_3)
        
        opt_4 = [0 for i in range(8)]
        for opt in v['option_4']:
            opt_4[allopt_4.index(opt)] = 1
        used_car_df['opt_etc'].append(opt_4)            

        used_car_df['description'].append(v['판매자설명'])
        used_car_df['price'].append(re.sub('[\D]','',v['price']))
    
    except Exception as e:
        print(k)
        raise Exception
        key_exceptions.append(k)
        continue
    

In [40]:
len(key_exceptions)

0

In [11]:
# for e in key_exceptions:
#     del used_car_data[e]

In [42]:
used_car_df = pd.DataFrame(used_car_df)

In [44]:
used_car_df.to_excel('used_car.xlsx',encoding='utf-8')

In [46]:
used_df.demoNo = used_df.demoNo.astype('str')

In [47]:
used_df = used_df.set_index('demoNo')

In [48]:
used_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7129 entries, 8121501822 to 5121101132
Data columns (total 26 columns):
title          7129 non-null object
sub_title      7129 non-null object
baegi          7093 non-null float64
year_old       7129 non-null object
color          6985 non-null object
driven_dist    7129 non-null int64
trans          7128 non-null object
fuel           7128 non-null object
mortgage       7129 non-null object
tax_to_pay     7129 non-null object
ch_purpose     6092 non-null object
ch_num         7129 non-null object
ch_owner       7129 non-null object
total_loss     6092 non-null float64
total_flood    6092 non-null float64
part_flood     6092 non-null float64
theft          6092 non-null float64
damaged        6092 non-null float64
damaging       6092 non-null float64
opt_prefer     7129 non-null object
opt_basic      7129 non-null object
opt_safe       7129 non-null object
opt_conven     7129 non-null object
opt_etc        7129 non-null object
description  

기본정보와 중고차 데이터의 매핑

In [49]:
car_base = pd.read_excel('car_base.xlsx')

In [50]:
car_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527 entries, 0 to 526
Data columns (total 7 columns):
model          527 non-null int64
brand          527 non-null object
seg            527 non-null object
title          527 non-null object
duration       527 non-null object
price_range    503 non-null object
recall_num     527 non-null int64
dtypes: int64(2), object(5)
memory usage: 32.9+ KB


In [51]:
model_suv_lst = [str(el) for el in list(car_base[car_base.seg == '소형SUV'].model)] # 소형suv의 model_id를 뽑자

In [52]:
len(model_suv_lst)

68

In [53]:
with open('model_used.json') as fp:
    model_used_mapping = json.load(fp)

In [54]:
len(model_used_mapping) # 다나와에서 가져온 526개 모델의 매핑 정보 담음

527

In [55]:
model_used_mapping['3341'] # 엠파크, 국민차 등 섞인상태

['8910001899',
 '9011502664',
 '8710002053',
 '9110003820',
 '8C10002716',
 '9011502097',
 '9110001819',
 '8C20001916',
 '9120002263',
 '9011001070',
 '9011000945',
 '8C10001295',
 '8121501330',
 '8710002046',
 '8121100039',
 '8111002759',
 '8810002469',
 '8111002255',
 '8111002192',
 '8101002509',
 '8101500875',
 '8091002005',
 '8071502570',
 '8071501209',
 '8061002389']

In [56]:
used_df['model_id'] = pd.Series([0 for _ in range(len(used_df))]) # used_df에 model_id 컬럼 명시적으로 추가해보자

In [57]:
for k,no_lst in model_used_mapping.items(): # 매핑된(다나와랑 연결되있는)경우 에 model_id 명시가능
    for no in no_lst:
        if no in used_df.index:
            used_df.loc[no,'model_id'] = str(k)

In [58]:
used_df.shape

(7129, 27)

In [61]:
used_df.head(1)

,title,sub_title,baegi,year_old,color,driven_dist,trans,fuel,mortgage,tax_to_pay,...,damaged,damaging,opt_prefer,opt_basic,opt_safe,opt_conven,opt_etc,description,price,model_id
demoNo,,,,,,,,,,,,,,,,,,,,,
8121501822,르노삼성 뉴SM5(신형),가솔린 2.0 RE,1998.0,2011,흰색,44038,자동,휘발유,정보없음/정보없음,-,...,NaN,NaN,"[1, 1, 1, 0, 1, 1, 0, 0]","[1, 1, 0, 1, 1, 1, 0, 1]","[1, 1, 1, 1, 1, 0, 1, 1]","[0, 1, 1, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","풀옵션, 안마시트,메모리시트, 파썬, 네비,후방,블박,상태최상인 차량입니다. 안녕하...",790,2112


In [62]:
used_df.to_excel('used_car_map.xlsx',encoding='utf-8')

In [64]:
used_suv_lst = []
for k,no_lst in model_used_mapping.items():
    if k in model_suv_lst:
        used_suv_lst += no_lst

In [65]:
len(used_suv_lst)

1284

In [66]:
valid_no = []
for no in used_suv_lst:
    if no in used_df.index:
        valid_no.append(no)

In [67]:
valid_idx = pd.Index(valid_no)

In [68]:
used_suv = used_df.loc[valid_idx]

In [69]:
used_suv.sample(3)

,title,sub_title,baegi,year_old,color,driven_dist,trans,fuel,mortgage,tax_to_pay,...,damaged,damaging,opt_prefer,opt_basic,opt_safe,opt_conven,opt_etc,description,price,model_id
8121502408,폭스바겐 티구안(1세대),2.0TDI블루모션 컴포트,2000.0,2013,흰색,107148,자동,경유,정보없음/정보없음,-,...,5755510.0,79290.0,"[1, 1, 0, 0, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 0, 1]","[1, 1, 1, 1, 1, 1, 1, 0]","[0, 1, 0, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","무사고,브라운시트,네비매립(200만원),후방카메라,열선시트",1490,2337
9011501861,현대 올뉴투싼(TL),디젤1.7 2WD 모던,1685.0,2016,흰색,28034,자동,경유,정보없음/정보없음,-,...,962050.0,1453980.0,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",NaN,2120,3250
8111001833,폭스바겐 티구안(1세대),2.0TDI블루모션 프리미엄,2000.0,2015,회색,36793,자동,경유,정보없음/정보없음,-,...,724200.0,0.0,"[1, 1, 0, 1, 1, 1, 0, 1]","[0, 1, 0, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 1, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",완무 1인신조 신차급 병적관리 오일류교체등 경정비완료되어 바로 타면 됩니다.,2540,2337


In [70]:
used_suv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 509 entries, 9011501978 to 8071501209
Data columns (total 27 columns):
title          509 non-null object
sub_title      509 non-null object
baegi          509 non-null float64
year_old       509 non-null object
color          498 non-null object
driven_dist    509 non-null int64
trans          509 non-null object
fuel           509 non-null object
mortgage       509 non-null object
tax_to_pay     509 non-null object
ch_purpose     413 non-null object
ch_num         509 non-null object
ch_owner       509 non-null object
total_loss     413 non-null float64
total_flood    413 non-null float64
part_flood     413 non-null float64
theft          413 non-null float64
damaged        413 non-null float64
damaging       413 non-null float64
opt_prefer     509 non-null object
opt_basic      509 non-null object
opt_safe       509 non-null object
opt_conven     509 non-null object
opt_etc        509 non-null object
description    418 non-null object
pri

In [71]:
used_suv.groupby('title').mean()[['price']].sort_values('price',ascending=False)

,price
title,
지프 랭글러(JL),5200.000000
랜드로버 레인지로버이보크,4590.000000
미니 컨트리맨(2세대),3750.000000
지프 랭글러(JK),3697.500000
랜드로버 디스커버리스포츠,3550.000000
벤츠 GLA클래스,3436.666667
지프 레니게이드,2552.500000
토요타 RAV4,2340.000000
아우디 Q3,2312.500000


In [72]:
used_suv.describe(include='object')

,title,sub_title,year_old,color,trans,fuel,mortgage,tax_to_pay,ch_purpose,ch_num,ch_owner,opt_prefer,opt_basic,opt_safe,opt_conven,opt_etc,description,model_id
count,509,509,509,498,509,509,509,509,413,509,509,509,509,509,509,509,418,509
unique,40,175,16,31,3,3,2,2,2,4,9,70,58,39,48,21,404,41
top,현대 올뉴투싼(TL),디젤 2WD 프레스티지,2016,흰색,자동,경유,정보없음/정보없음,-,없음,0회,1회,"[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",무사고,3250
freq,61,15,134,208,507,415,494,507,298,285,147,199,232,262,272,394,6,61


In [73]:
used_suv.title.value_counts()

현대 올뉴투싼(TL)       61
기아 스포티지4세대        56
기아 더뉴스포티지R        51
현대 투싼ix(LM)       38
기아 스포티지R          35
쌍용 티볼리            31
폭스바겐 티구안(1세대)     30
현대 뉴투싼ix(LM)      23
쌍용 뉴코란도C          23
쉐보레/대우 트랙스        19
미니 컨트리맨(1세대)      16
쉐보레/대우 더뉴트랙스      15
르노삼성 QM3          12
쌍용 티볼리에어          10
기아 니로              8
쌍용 코란도C            8
현대 코나              8
쌍용 티볼리아머           7
기아 스토닉             7
지프 컴패스(1세대)        6
BMW X1(E84)        5
닛산 쥬크              4
아우디 Q3             4
지프 레니게이드           4
쌍용 뉴스타일코란도C        4
지프 랭글러(JK)         4
벤츠 GLA클래스          3
미니 컨트리맨(2세대)       3
닛산 캐시카이            2
푸조 2008(1세대)       2
포드 이스케이프(2세대)      1
랜드로버 레인지로버스포츠      1
랜드로버 레인지로버이보크      1
지프 랭글러(JL)         1
기아 스포티지            1
랜드로버 디스커버리스포츠      1
르노삼성 뉴QM3          1
피아트 500X           1
토요타 RAV4           1
포드 쿠가(2세대)         1
Name: title, dtype: int64